# Shadow Tomography

In this notebook we give an overview of a method, introduced in [1] and made more hardware efficient in [2], of predicting functions of a quantum state, without relatively many measurements or performing full tomography. The motivation is that for certain characterization tasks, we may not need a full description of a quantum state to predict observables of interest, avoiding the prohibitive sampling costs of state tomography at scale. First we'll import some tools we'll use.

In [1]:
# importing basic tools
import numpy as np
from functools import reduce
from itertools import combinations

# importing qiskit
from qiskit import *
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute

from copy import deepcopy
from qiskit.quantum_info import state_fidelity, purity
from qiskit.providers.aer import noise

from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error


# Load IBMQ account
# provider = IBMQ.load_account()
# provider.backends()

### Introduction

Predicting functions of a quantum state, such as fidelity and entanglement, are important tasks for engineering quantum systems. Due to the probabilistic nature of measurements in quantum mechanics, predictions are made through many measurements of a given state, as in state tomography. As these systems grow in size, say a quantum computer with n qubits, obtaining a full description of these states requires sampling that scales exponentially in n. 

In [1], it was shown that this full description can be excessive for the intended use. Instead, we are able to predict the intended expectation values $o_{i}$ of an exponentially large set of M observables $O_{i}$:

\begin{equation*}
o_{i}(\rho) = tr(O_{i}\rho)
\end{equation*}

with a polynomial number of samples (i.e N = $O(log(M))$, using a method dubbed "shadow tomography". This method was equipped with a more efficient implementation and rigorous statistical convergence guarantees in [2], and is the procedure that will be followed here.

### Procedure

Assume the result of a quantum computation, $\rho$ is some $d = 2^{n}$ dimensional unknown quantum state. To predict values of functions of this state, we repeatedly perform the following procedure: apply a random unitary U (selected from a fixed ensemble) and perform a single-shot computational basis measurement. Applying U to $\rho$ gives $U \rho U^{\dagger}$, so from the measurement outcomes $|b\rangle$, we can construct a classical description as $U^{\dagger} |b\rangle \langle b|U$ that is essentially the collapsed form of the full quantum state. Below is a function to help us get the necessary vector from a measurement output.



In [4]:
def bitToVector(bitstring):
    
    """
    Create vectorized version of a bitstring.
    """
    n = len(bitstring)
    
    b = np.zeros(2**n)
    b[int(bitstring,2)] = 1
    b.shape = (2**n,1)
    
    return b



Viewing the mapping of the original state to the average classical description (over the unitary and sample distributions) as a quantum channel, we have:

\begin{equation*}
\mathbb{E}[U^{\dagger} |\hat{b}\rangle \langle \hat{b}|U] = M(\rho) \implies \rho = \mathbb{E} [M^{-1}(U^{\dagger} |\hat{b}\rangle \langle \hat{b}|U)]
\end{equation*}

For this notebook we take a look at two unitary distributions that affect the form of M, random n-qubit Clifford circuits, and tensor products of random single-qubit Clifford circuits (measuring in a random Pauli basis). M comes out to be $M_{n}^{-1} = (2^{n}+1)\rho - \mathbb{I}$ and $M_{P}^{-1} = \bigotimes_{i=1}^{n} M_{1}^{-1} = \bigotimes_{i=1}^{n} 3\rho - \mathbb{I}$ for n-qubit Cliffords and random Pauli measurements, respectively. Next we give a couple of functions to compute this map and produce a snapshot.

In [82]:
def invM(rho,n):
    
    """
    Compute inverse channel operation for n-qubit random Clifford Circuits
    """
    
    rho = (2**n+1)*rho - np.eye(2**n)
     
    return rho

def getSnapshot(state,Udg,n):
    
    """
    Compute classical snapshot of a quantum state from its measurement outcome.
    """
    
    snapshot = Udg.dot(state)     
    snapshot = snapshot.dot(snapshot.conj().T)
    snapshot = invM(snapshot,n)
    
    return snapshot
        

This mapping, while not completely positive, is designed to be applied to the single measurement classical description of the quantum state to obtain the classical snapshot $\hat{\rho} = M^{-1}(U^{\dagger} |\hat{b}\rangle \langle \hat{b}|U)$ that exactly reproduces $\rho$ in expectation, $\mathbb{E}[\hat{\rho}] = \rho $. So for N measurements, we produce N snapshots which form an array called the classical shadow of $\rho$:

\begin{equation*}
S(\rho;N) = \lbrace\hat{\rho}_{i} = U^{\dagger}_{i} |\hat{b_{i}}\rangle \langle \hat{b_{i}}|U_{i}\rbrace_{i = 1}^{N}
\end{equation*}

From this array, we break it into K equally-sized chunks to form K empirical mean estimators of the form:

\begin{equation*}
\hat{\rho}_{(k)} = \frac{1}{\lfloor N/K\rfloor} \sum_{i=(k-1)\lfloor N/K\rfloor+1}^{k\lfloor N/K\rfloor}{\hat{\rho}_{i}}
\end{equation*}

In [5]:
def chunks(lst, n):
    
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

With these estimators we can perform median of means estimation of linear functions of the original state, i.e. given an observable $O$ and estimators $\hat{\rho}_{(k)}$, the expectation can be estimated as:

\begin{equation*}
\ o(\rho) = tr(O\rho) \approx \hat{o}(N,K) = median\lbrace tr(O \hat{\rho}_{(i)}) \rbrace_{i=1}^{K}
\end{equation*}

Now we are equipped to implement this in Qiskit. First, let's consider the full n-qubit Clifford case.

### Classical Shadow Generation

First lets initialize some parameters, and the circuit to produce the state with which we want to predict functions. As in [2] we'll study the GHZ state, starting with 2 qubit case. We'll look at fidelity as an example, so we'll need the real statevector as well.

In [33]:
def generateGHZ(circ):
    """
    Build circuit to gneerate GHZ state for N-qubits
    """
    
    qubits = circ.qubits
    n = len(qubits)
    
    if n == 1:
        print('There is only one qubit')
        return
    else:
        circ.h(0)
        
        for q in qubits[1:]:
            circ.cx(0,q)
        
        return
    

In [17]:
# initialize circuit parameters 
n = 2
backend = Aer.get_backend('qasm_simulator')

# initialize shadow
N = 500
K = 100

# initialize circuit
ghz = QuantumCircuit(n)

# build circuit
generateGHZ(ghz)

job = qiskit.execute(ghz, Aer.get_backend('statevector_simulator'))
psi = job.result().get_statevector(ghz)

# print the ciruit
print(psi,'\n\n',ghz)

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j] 

      ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘


Now, we'll create shadow tomography circuits from random Cliffords and random Pauli measurements to compare later on.

##### Shadow from Random Cliffords

Here we define a function to perform random Clifford measurements.

In [35]:
def n_clifford_circuit(n):
    
    # generate random clifford 
    U = qiskit.quantum_info.random_clifford(n)
    G = qiskit.extensions.UnitaryGate(U)
    Udg = G.to_matrix().conj().T

    qc = QuantumCircuit(n,n)
    q = qc.qubits
    c = qc.clbits
    # apply gates
    qc.unitary(G,q)
    qc.measure(q,c)
    
    
    return Udg, qc


Now we can finally generate the classical shadow of our Bell state.

In [148]:
def get_clifford_shadow(circ,N,nqubits):
    
    shadow_clifford = []

    for _ in range(N):

        Udg, mc = n_clifford_circuit(nqubits)
        qc = circ + mc

        # create classical snapshot
        job = qiskit.execute(qc, backend,shots=1, memory=True)
        result = job.result()
        memory = result.get_memory(qc)[0]

        b = bitToVector(memory)
        snapshot = getSnapshot(b,Udg,nqubits)

        shadow_clifford += [snapshot]
        
    return shadow_clifford


In [111]:
shadow_clifford = get_clifford_shadow(ghz,N,n)

In [112]:
shadow_clifford[0]

array([[ 2.50000000e-01-2.95197503e-17j, -2.95197503e-17-1.25000000e+00j,
         1.25000000e+00-2.95197503e-17j,  2.95197503e-17+1.25000000e+00j],
       [-2.95197503e-17+1.25000000e+00j,  2.50000000e-01+2.95197503e-17j,
        -2.95197503e-17+1.25000000e+00j, -1.25000000e+00-2.95197503e-17j],
       [ 1.25000000e+00-2.95197503e-17j, -2.95197503e-17-1.25000000e+00j,
         2.50000000e-01-2.95197503e-17j,  2.95197503e-17+1.25000000e+00j],
       [ 2.95197503e-17-1.25000000e+00j, -1.25000000e+00-2.95197503e-17j,
         2.95197503e-17-1.25000000e+00j,  2.50000000e-01+2.95197503e-17j]])

##### Shadow from Random Pauli Measurements

Next, we define a similar function for Pauli measurements.

In [90]:
def n_pauli_circuit(n):
    
    # initialize circuit
    qc = QuantumCircuit(n,n)
    q = qc.qubits
    c = qc.clbits
    
    qbases = {}

    # apply random single qubit measurements
    for qbit in range(n):

        which_measure = np.random.rand(1,1)

        # perform x basis measurement if less than 1/3
        if (which_measure < 1/3):
            qc.h(qbit)
            qbases[qbit] = 0

        # perform y basis measurement if between 1/3 and 2/3
        elif (which_measure >= 1/3) & (which_measure < 2/3):
            qc.sdg(qbit)
            qc.h(qbit)
            qbases[qbit] = 1

        # otherwise perform z basis measurement
        else:
            qbases[qbit] = 2

    qc.measure(q,c)
    
    return qbases, qc

In [176]:
def get_pauli_shadow(circ,N,nqubits):
    
    shadow_pauli = []

    H = np.array([[1,1],[1,-1]])/np.sqrt(2)
    S = np.array([[1,0],[0,1j]])
    I = np.eye(2)

    Udgs = [H,S.dot(H),I]

    for _ in range(N):

        qbases, mc = n_pauli_circuit(nqubits)
        qc = circ + mc

        # create classical snapshot
        job = qiskit.execute(qc, backend ,shots=1, memory=True)
        result = job.result()
        memory = result.get_memory(qc)[0]

        # reverse order of bits to loop through for applying M
        memory = memory[::-1]

        # initialize snapshot tensor
        snapshot = 1
        for bit in range(len(memory)):

            b = bitToVector(memory[bit])
            Udg = Udgs[qbases[bit]]
            subsnapshot = getSnapshot(b,Udg,1)
            snapshot = np.kron(snapshot,subsnapshot)

        shadow_pauli += [snapshot]


    return shadow_pauli



In [119]:
shadow_pauli = get_shadow_pauli(ghz,N,n)

In [121]:
shadow_pauli[0]

array([[ 1., -0., -0.,  0.],
       [-0., -2.,  0.,  0.],
       [-0.,  0., -2.,  0.],
       [ 0.,  0.,  0.,  4.]])

##### Median of Means Prediction

Linear Case

We have generated shadows for both full system Cliffords and local Pauli measurements. As a simple example of the median of means protocol, we'll look at fidelity of the state. Since this would be a global variable, local Paulis aren't suited to the task, so we turn to the Clifford shadow. Let's create the estimators of our state.

In [138]:
def get_estimators(shadow,N,K):
    
    # split shadow into K equally sized parts
    estimators = []
    shades = list(chunks(shadow,int(N/K)))

    # create estimators
    for shade in shades:
        estimators += [reduce(np.add,shade)/np.floor(N/K)]
        
    return estimators

In [139]:
shadow = shadow_clifford
estimators = get_estimators(shadow,N,K)

Now, using the state fidelity function from qiskit, and the simulated statevector from earlier, we can get a prediction of the true fidelity by evaluating the function on each of the estimators and taking the real portion of the median value.

In [146]:
def get_fidelity_prediction(estimators,psi):
    
    predictions = [state_fidelity(psi,rho,validate=False) for rho in estimators]
    prediction = np.real(np.median(predictions))
    
    return prediction


In [147]:
prediction = get_fidelity_prediction(estimators,psi)
prediction

0.9999999999999991

The implementation we have here is merely instructive, so is not terribly efficient as we're still storing large matrices. This can be improved significantly as the estimators are sparse. However we can check scaling in a few qubits to check performance holding our number of samples constant. 

In [178]:
nqubits = [i for i in range(2,10,1)]

for nq in nqubits:
    
    # initialize circuit
    ghz = QuantumCircuit(nq)

    # build circuit
    generateGHZ(ghz)

    job = qiskit.execute(ghz, Aer.get_backend('statevector_simulator'))
    psi = job.result().get_statevector(ghz)
    
    shadow = get_clifford_shadow(ghz,N,nq)
    estimators = get_estimators(shadow,N,K)
    prediction = get_fidelity_prediction(estimators,psi)
    
    print(str(nq)+': ', prediction)

2:  0.9999999999999992
3:  1.0249999999999977
4:  0.9124999999999979
5:  0.8562499999999975
6:  1.0312499999999964
7:  0.8140624999999969
8:  1.0078124999999956
9:  1.0039062499999947


Considering we varied nothing except the number of qubits in the system, the performance is decent. Also, values can be greater than 1 as we are using an unphysical estimator: there is a variance around the average, but we can mitigate this by optimizing chunk and sample size through a procedure like grid search. Let's see how the Pauli shadow performs in comparison.

In [177]:
nqubits = [i for i in range(2,10,1)]

for nq in nqubits:
    
    # initialize circuit
    ghz = QuantumCircuit(nq)

    # build circuit
    generateGHZ(ghz)

    job = qiskit.execute(ghz, Aer.get_backend('statevector_simulator'))
    psi = job.result().get_statevector(ghz)
    
    shadow = get_pauli_shadow(ghz,N,nq)
    estimators = get_estimators(shadow,N,K)
    prediction = get_fidelity_prediction(estimators,psi)
    
    print(str(nq)+': ', prediction)

2:  0.9249999999999996
3:  1.024999999999999
4:  0.8499999999999992
5:  0.4531249999999992
6:  0.47968749999999916
7:  0.2890624999999992
8:  0.24296874999999915
9:  0.30429687499999886


As seen above, the prediction fails around a >5 qubit system size. This indicates, as we have guessed, that while the Pauli shadow is easier to compute (both classically and on a quantum computer due to te lack of entangling operations), it is more suited for local subsystem predictions involving just a few qubits. Some useful examples are entanglement entropies and simulation of localized Hamiltonian physics. Of note is that the former is actually a transcendental function of a product of density matrices, so non-linear functions can also be predicted using this method. A continuation of this notebook will cover how this can further be adapted for non-linear prediction. 

# References

[1] S. Aaronson. Shadow Tomography of Quantum States (https://arxiv.org/abs/1711.01053) 

[2] H. Huang, R. Kueng, J. Preskill (https://arxiv.org/abs/2002.08953)